In [33]:
import plotly
import plotly.plotly as py
import numpy as np
from plotly.graph_objs import *
import os
import requests
from os import environ

requests.packages.urllib3.disable_warnings()

plotly.tools.set_credentials_file(username='pusiol', api_key='m6iurl7f89')

In [34]:
default_filepath = '/media/Gui2/thermix/ARTraining/trained_models/thermix_28a/output_thermix_28a.file'
filepath = environ.get("output_filepath",default_filepath)

model_output_dir = os.path.dirname(filepath)

model_name = os.path.basename(filepath)[7:].split(".")[0]

In [35]:
with open(filepath,'r') as f:
    # get all lines in file
    lines = f.readlines()

    # remove all those weird chars
    newlines = []
    for line in lines:
        newline = line.replace('\x08',' ')
        newlines.append(filter(None, newline.split('  ')))

    training_batch_count = None
    testing_batch_count = None
    for line in newlines:
        if training_batch_count and testing_batch_count:
            break;
            
        for s in line:
            if training_batch_count and testing_batch_count:
                break;

            if not training_batch_count and "stochastic gradient descent with miss rate" in s:
                # txt = "% at 01 / 110 ("
                txt = s[s.index("% at "):]
                txt = txt[:txt.index("(")]
                
                # training_batch_count = 110
                training_batch_count = int(txt.split("/")[1][1:-1])
            
            if not testing_batch_count and "=> with miss rate" in s:
                # txt = "at 01 / 24 ("
                txt = s[s.index("% at "):]
                txt = txt[:txt.index("(")]
                
                # testing_batch_count = 110
                testing_batch_count = int(txt.split("/")[1][1:-1])
            
    if not training_batch_count or not testing_batch_count:
        raise "malformed file"
    #now search for 75 / 75 (training) y 16 / 16 (validation)
    training_error_text = '%d / %d' % (training_batch_count, training_batch_count)
    validation_error_text = '%d / %d' % (testing_batch_count, testing_batch_count)

    print training_error_text
    print validation_error_text
    
    training_error_lines = []
    validation_error_lines = []
    for a in newlines:
        for s in a:
            # Discard line when not contains 'rate' to avoid error like the following:
            # validation_error_text = "18 / 18" and s = "- compute mean activity 1818 / 18267"
            if 'rate' not in s:
                continue
                
            if training_error_text in s:
                training_error_lines.append(s)
            if validation_error_text in s:
                validation_error_lines.append(s)

    # get matrix for validation loss
    v_loss = []
    for v_data in validation_error_lines:
        index_rate = v_data.index('rate')
        v_loss.append(float(v_data[index_rate+4:index_rate+10].replace('%','').replace(' ','')))

    # get matrix for training loss
    t_loss = []
    for t_data in training_error_lines:
        index_rate = t_data.index('rate')
        t_loss.append(float(t_data[index_rate+4:index_rate+10].replace('%','').replace(' ','')))

    # export loss values so later can be gathered from outside the notebook
    np.savetxt(os.path.join(model_output_dir, model_name+"_validationLoss.out"), v_loss, fmt='%.2f')
    np.savetxt(os.path.join(model_output_dir, model_name+"_trainingLoss.out"), t_loss, fmt='%.2f')

34 / 34
9 / 9


In [36]:
#plot data
data = []
Ys = [('validation sparse softmax cross entropy loss', v_loss), ('training sparse softmax cross entropy loss', t_loss)]
for title, _Y in Ys:
    # t = 1 -> validation data
    _X = range(1,len(_Y)+1) # _X = epoch number
    mom = Scatter(x=_X,y=_Y, mode ='lines+markers', name =title ,line = dict(width = 4))
    data.append(mom)

layout = Layout(title='%s loss function'%model_name, xaxis = dict(title = 'epoch'), yaxis = dict(title = 'miss rate'), )

# Plot and embed in ipython notebook!
fig = dict(data=data, layout=layout)

#upload to guido's account
py.iplot(fig, filename="lastest_trained_model_info")
py.iplot(fig, filename="%s_trained_model_info"%model_name)